In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import librosa
import pandas as pd
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np

Read the CSV file into a DataFrame

In [ ]:
csv_file_path = 'QDAT_Quran Recitation.csv'
df = pd.read_csv(csv_file_path)

In [ ]:
df

,title,Separate tide,The tight noon,Concealment,Target,Age,Gender
0,S0_1,0,1.0,1,0,14,1
1,S0_2,0,1.0,1,0,14,1
2,S0_3,0,1.0,1,0,14,1
3,S0_4,0,1.0,1,0,14,1
4,S0_5,0,1.0,1,0,14,1
...,...,...,...,...,...,...,...
1505,S160_6,1,1.0,1,1,21,1
1506,S160_7,1,1.0,1,1,21,1
1507,S160_8,1,1.0,1,1,21,1
1508,S160_9,1,1.0,1,1,21,1


In [ ]:
audio_folder = 'FINAL SOUND/FINAL SOUND'

In [ ]:
audio_files_df = pd.DataFrame({'audio_files': [f for f in os.listdir(audio_folder) if f.lower().endswith('.wav')]})

df = pd.merge(df, audio_files_df, left_index=True, right_index=True)

In [ ]:
base_url = "FINAL SOUND/FINAL SOUND"

In [ ]:
df['audio_links'] = base_url + '/' + df['audio_files']

In [ ]:
directory_path = 'Quran_e_pak/'

In [ ]:
os.makedirs(directory_path, exist_ok=True)

In [ ]:
df.to_csv(os.path.join(directory_path, 'QDAT_Quran Recitation.csv'), index=False)

In [ ]:
df.head(3)

,title,Separate tide,The tight noon,Concealment,Target,Age,Gender,audio_files,audio_links
0,S0_1,0,1.0,1,0,14,1,S111_10.wav,/content/drive/MyDrive/AI Project/P1/Quran_e_p...
1,S0_2,0,1.0,1,0,14,1,S11_8.wav,/content/drive/MyDrive/AI Project/P1/Quran_e_p...
2,S0_3,0,1.0,1,0,14,1,S13_10.wav,/content/drive/MyDrive/AI Project/P1/Quran_e_p...


In [ ]:
import librosa
import numpy as np

def extract_features(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    features = np.concatenate([mfccs.mean(axis=1), chroma.mean(axis=1), spectral_contrast.mean(axis=1)])

    return features
df['features'] = df['audio_links'].apply(extract_features)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['features'].tolist(), df['Target'], test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)


In [ ]:
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


Accuracy: 0.75
Classification Report:
              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       1.00      0.33      0.50         3

    accuracy                           0.75         8
   macro avg       0.86      0.67      0.67         8
weighted avg       0.82      0.75      0.71         8



In [ ]:
import joblib

In [ ]:
model_path = 'model.joblib'
joblib.dump(model, model_path)

['/content/drive/MyDrive/AI Project/P1/Quran_e_pak/Zunaisha_model.joblib']